# Laboratorium 3

In [1]:
import numpy as np
from scipy.special import binom

In [2]:
class CRR:
    def __init__(self, S0, d, u, r, T):
        if not (d < 1 + r < u):
            raise ValueError('in CRR model it must be d < 1 + r < u')
        self.S0 = S0
        self.d = d
        self.u = u
        self.r = r
        self.T = T
        self.p = (1 + r - d)/(u - d)
        
    def get_S_trajectory(self, trajectory):
        t = np.ndarray.copy(trajectory)
        t = np.insert(t, 0, 1.0, axis=0)
        return np.cumprod(t) * self.S0
    
    def get_rational_price(self, option, trajectory=None):
            if trajectory is None:
                C0 = self.p * self.get_rational_price(option, np.array([self.u]))
                C0 += (1 - self.p) * self.get_rational_price(option, np.array([self.d]))
                C0 /= 1 + self.r
                return C0
            elif len(trajectory) < self.T:
                tu = np.append(np.ndarray.copy(trajectory), self.u)
                td = np.append(np.ndarray.copy(trajectory), self.d)
                Ct = self.p * self.get_rational_price(option, tu) 
                Ct += (1 - self.p) * self.get_rational_price(option, td)
                Ct /= 1 + self.r
                return Ct
            else:
                S_trajectory = self.get_S_trajectory(trajectory)
                return option.get_final_price(S_trajectory)

In [3]:
class Option:
    def __init__(self, K):
        self.K = K
    
    def get_final_price(self, S_trajectory):
        pass
   

class CallOption(Option):
    def __init__(self, K):
        super().__init__(K)
        
    def get_final_price(self, S_trajectory):
        return max(S_trajectory[-1] - self.K, 0.0)
    
    
class PutOption(Option):
    def __init__(self, K):
        super().__init__(K)
        
    def get_final_price(self, S_trajectory):
        return max(self.K - S_trajectory[-1], 0.0)

    
class SomeOtherOption(Option):
    def __init__(self, K=0.0):
        super().__init__(K)
        
    def get_final_price(self, S_trajectory):
        return np.max(S_trajectory)

In [4]:
crr = CRR(100, 0.8, 1.3, 0.1, 10)

## Call Option

In [5]:
call = CallOption(90)
call_price = crr.get_rational_price(call)
print(f'Call Option rational price: {call_price:.2f}')

Call Option rational price: 66.97


In [6]:
C0 = 0.0
for i in range(crr.T + 1):
    C0 += binom(crr.T, i) * (crr.p**i) * ((1 - crr.p)**(crr.T - i)) * max(crr.S0 * crr.u**i * crr.d**(crr.T - i) - call.K, 
                                                                          0)
C0 /= (1 + crr.r)**crr.T
print(f'Call Option (theoretical) rational price: {C0:.2f}')

Call Option (theoretical) rational price: 66.97


## Put Option

In [7]:
put = PutOption(90)
put_price = crr.get_rational_price(put)
print(f'Put Option rational price: {put_price:.2f}')

Put Option rational price: 1.67


In [8]:
C0 = 0.0
for i in range(crr.T + 1):
    C0 += binom(crr.T, i) * (crr.p**i) * ((1 - crr.p)**(crr.T - i)) * max(put.K - crr.S0 * crr.u**i * crr.d**(crr.T - i), 
                                                                          0)
C0 /= (1 + crr.r)**crr.T
print(f'Put Option (theoretical) rational price: {C0:.2f}')

Put Option (theoretical) rational price: 1.67


## Some Other Option

$X = \max_{t \in \mathcal{T}} \{S_t\}$

In [9]:
soo = SomeOtherOption()
soo_price = crr.get_rational_price(soo)
print(f'Some Other Option rational price: {soo_price:.2f}')

Some Other Option rational price: 116.88
